<center><h1 style="color:#D4AF37"> AEMR Data Analysis </h1>

The American Energy Market Regulator (AEMR) is responsible for looking after the
United States of America’s domestic energy network. The regulator’s responsibility is to
ensure that America’s energy network remains reliable with minimal disruptions, which
are known as outages. 

There are four key types of outages:

● Consequential

● Forced 

● Opportunistic 

● Planned 

Recently, the AEMR management team has been increasingly aware of a large number
of energy providers that submitted outages over the 2016 and 2017 calendar years. The
management team has expressed a desire to have the following two areas of concern
addressed:

<b> A) Energy Stability and Market Outages
    <p>
B) Energy Losses and Market Reliability </b>

The analysis aims to address these two immediate areas of concern. 

In [158]:
!pip install ipython-sql
!pip install prettytable==3.12

In [159]:
import requests
from IPython.core.magic import register_line_magic
from IPython.display import HTML
import sqlite3

@register_line_magic
def load_sqlite_db(url):
    response = requests.get(url)

    if response.status_code == 200:
        with open('temp_db_file.db', 'wb') as file:
            file.write(response.content)
        print('SQLite database file downloaded successfully.')
    else:
        print('Failed to download the SQLite database file.')

sqlite_db_url = 'https://raw.githubusercontent.com/chrishuisb1990/practice_datasets/main/AEMR.db'

%load_sqlite_db $sqlite_db_url

%load_ext sql

%sql sqlite:///temp_db_file.db

%config SqlMagic.style = '_DEPRECATED_DEFAULT'

SQLite database file downloaded successfully.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


<h3 style="color:#D4AF37"> Analyzing Energy Stability & Market Outages </h3>

Energy stability is one of the key themes the AEMR management team cares about. To ensure energy security and reliability, AEMR needs to understand the following:
<p>

<li> What are the most common outage types and how long do they tend to last? 
<li> How frequently do the outages occur? 
<li> Are there any energy providers which have more outages than their peers which may be indicative of being unreliable? 
    
<p>

Note: Throughout the entire case study, we are analyzing Outages where Status = Approved.

<b> Counting the number of valid Outage events according to their reason over 2016 and 2017. 

In [160]:
%%sql
SELECT
    COUNT(*) AS 'Total_Number_Outages', 
    Outage_Reason, 
    Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY 
    Year, 
    Outage_Reason
;

 * sqlite:///temp_db_file.db
Done.


Total_Number_Outages,Outage_Reason,Year
181,Consequential,2016
1264,Forced,2016
106,Opportunistic Maintenance (Planned),2016
380,Scheduled (Planned),2016
127,Consequential,2017
1622,Forced,2017
102,Opportunistic Maintenance (Planned),2017
320,Scheduled (Planned),2017


Forced outages occur most frequently.

<b> Determining the total number of outages across different months and years.

In [162]:
%%sql
SELECT Year, Month, count(EventID) as Total_Number_Outages
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Year, Month
ORDER BY Year, Month

 * sqlite:///temp_db_file.db
Done.


Year,Month,Total_Number_Outages
2016,1,191
2016,2,227
2016,3,136
2016,4,134
2016,5,174
2016,6,168
2016,7,147
2016,8,194
2016,9,124
2016,10,124


Forced outage events increased in 2017 compared to 2016. 

<b> Calculating the Total Number Outage Events and the Average Duration in days for each Participant Code and Outage Type over the 2016 and 2017 period where Status = Approved. 

In [165]:
%%sql
SELECT 
    Participant_Code, 
    Outage_Reason, 
    Year, 
    COUNT(EventID) AS Total_Number_Outage_Events, 
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) AS Average_Outage_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY 
    Participant_Code, 
    Outage_Reason, 
    Year
ORDER BY 
    Total_Number_Outage_Events DESC  
LIMIT 5
;

 * sqlite:///temp_db_file.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days
AURICON,Forced,2017,490,0.07
GW,Forced,2016,317,0.38
GW,Forced,2017,227,1.06
AURICON,Forced,2016,208,0.07
AUXC,Forced,2016,206,0.08


This reveals participants who are having many outages, as well as participants who have been offline for the longest durations.

<b> Classifying each participant code as either `High Risk`, `Medium Risk` or `Low Risk` in a column called `Risk_Classification`</u> that is based off their Average Outage Duration Time.

In [172]:
%%sql
WITH Outage_Days AS 
(
SELECT 
    Participant_Code, 
    Outage_Reason, 
    Year, 
    COUNT(EventID) AS Total_Number_Outage_Events,
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) AS Average_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY 
    Participant_Code, 
    Outage_Reason, 
    Year
ORDER BY 
    Total_Number_Outage_Events DESC
)

SELECT *, 
CASE 
    WHEN Average_Duration_In_Days > 1 THEN 'High Risk'
    WHEN Average_Duration_In_Days BETWEEN 0.5 AND 1 THEN 'Medium Risk'
    WHEN Average_Duration_In_Days BETWEEN 0 AND 0.5 THEN 'Low Risk'
    ELSE 'N/A'
END AS Risk_Classification
FROM Outage_Days
LIMIT 4   
;

 * sqlite:///temp_db_file.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Duration_In_Days,Risk_Classification
AURICON,Forced,2017,490,0.07,Low Risk
GW,Forced,2016,317,0.38,Low Risk
GW,Forced,2017,227,1.06,High Risk
AURICON,Forced,2016,208,0.07,Low Risk


Redefining the risk categories as follows - 
<b>
<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day) OR the Total Number of Outage Events > 20 </li>
<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours OR the Total Number of Outage Events is Between 10 and 20 </li>
<li> Low Risk - On average, the participant is unavailable for less than 12 Hours OR the Total Number of Outage Events < 10 </li> 
<li> If Outage Type is not forced, then N/A
<b>

<b> Classifying outages based on the new classification criteria.

In [173]:
%%sql
WITH Outage_Days AS 
(
SELECT 
    Participant_Code, 
    Outage_Reason, 
    Year, 
    count(EventID) AS Total_Number_Outage_Events,
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) AS Average_Outage_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY 
    Participant_Code, 
    Outage_Reason, 
    Year
)

SELECT *,
CASE 
    WHEN Outage_Reason NOT LIKE 'Forced' THEN 'N/A'
    WHEN (Average_Outage_Duration_In_Days > 1) OR (Total_Number_Outage_Events > 20) THEN 'High Risk'
    WHEN (Average_Outage_Duration_In_Days > 0.5) OR (Total_Number_Outage_Events BETWEEN 10 AND 20) THEN 'Medium Risk'
    ELSE 'Low Risk'
END AS Risk_Classification
FROM Outage_Days
ORDER BY 
    Participant_Code, 
    Average_Outage_Duration_In_Days DESC
LIMIT 5
;

 * sqlite:///temp_db_file.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
AURICON,Scheduled (Planned),2016,46,1.89,N/A
AURICON,Scheduled (Planned),2017,45,1.45,N/A
AURICON,Opportunistic Maintenance (Planned),2016,3,0.33,N/A
AURICON,Consequential,2017,42,0.21,N/A
AURICON,Consequential,2016,41,0.13,N/A


<h3 style="color:#D4AF37"> Energy Losses & Market Reliability </h3>

Exploring the following questions : 
<li> Of the outage types in 2016 and 2017, what percent were Forced Outage(s)?
<li> What was the average duration for a forced outage during both 2016 and 2017? Have we seen an increase in the average duration of forced outages?
<li> Which energy providers tended to have the largest number of forced outages?

<b> Calculating the proportion of Forced Outages that have occurred over the 2016 - 2017 Period.

In [175]:
%%sql
WITH Total_Outages AS
(
SELECT 
    Year, 
    COUNT(EventID) AS Total_Number_Outage_Events
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Year
),

Forced_Outages AS 
(
SELECT 
    Year, 
    COUNT(EventID) AS Total_Number_Forced_Outage_Events
FROM AEMR_Outage_Table
WHERE Outage_Reason = 'Forced' AND Status = 'Approved'
GROUP BY Year
)
	
SELECT 
    Total_Number_Outage_Events, 
    Total_Number_Forced_Outage_Events,
    ROUND((Total_Number_Forced_Outage_Events*1.0 / Total_Number_Outage_Events)*100,2) AS Pct_Outage_Forced
FROM Total_Outages
JOIN Forced_Outages ON 
    Total_Outages.Year = Forced_Outages.Year
;

 * sqlite:///temp_db_file.db
Done.


Total_Number_Outage_Events,Total_Number_Forced_Outage_Events,Pct_Outage_Forced
1931,1264,65.46
2171,1622,74.71


<b> Determining the Total Number of Outages, Total Duration In Days and Total Energy Lost of all valid Outages for each Participant and Facility codes, sorted by Total Energy Lost in descending order and Ordered by the Year.

In [177]:
%%sql
SELECT 
    COUNT(EventID) AS Total_Number_Outages,
    ROUND(SUM(ABS(JULIANDAY(End_Time)-JULIANDAY(Start_Time))),2) AS Total_Duration_In_Days,
    ROUND(SUM(Energy_Lost_MW),2) AS Total_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY 
    Participant_Code, 
    Facility_Code,
    Outage_Reason, 
    Year
ORDER BY Total_Duration_In_Days DESC, Year DESC
LIMIT 5; 

 * sqlite:///temp_db_file.db
Done.


Total_Number_Outages,Total_Duration_In_Days,Total_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
70,482.58,7499.28,Scheduled (Planned),MELK,MELK_G7,2017
177,404.15,10285.4,Forced,MELK,MELK_G7,2017
85,392.25,9668.79,Scheduled (Planned),MELK,MELK_G7,2016
227,240.69,19326.56,Forced,GW,BW1_GREENWATERS_G2,2017
45,199.4,6450.0,Scheduled (Planned),GW,BW1_GREENWATERS_G2,2016


<b> Calculating the Average Duration In Days and Average Energy Lost of all valid Forced Outages for each Participant and Facility codes sorted by Average Energy Lost and Ordered by the Year. <b>

In [178]:
%%sql
SELECT
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) AS Avg_Duration_In_Days,
    ROUND(AVG(Energy_Lost_MW),2) AS Avg_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved' 
AND Outage_Reason = 'Forced'
GROUP BY
    Participant_Code,
    Facility_Code,
    Year 
ORDER BY Avg_Energy_Lost DESC, Year DESC
;

 * sqlite:///temp_db_file.db
Done.


Avg_Duration_In_Days,Avg_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
1.11,149.0,Forced,COLLGAR,COLLGAR_WF1,2016
0.04,141.21,Forced,PMC,PMC_AG,2017
0.49,131.78,Forced,PMC,PMC_AG,2016
0.83,87.71,Forced,MELK,MELK_G7,2016
1.06,85.14,Forced,GW,BW1_GREENWATERS_G2,2017
0.25,80.0,Forced,TSLA_MGT,TESLA_PICTON_G1,2016
0.38,76.23,Forced,KORL,KORL_GT3,2016
1.22,72.61,Forced,PJRH,PJRH_GT11,2016
0.84,67.21,Forced,PJRH,PJRH_GT11,2017
1.38,61.93,Forced,COLLGAR,COLLGAR_WF1,2017


<b> Calculating the Average Energy Lost and Total Energy Lost for each Facility Code and Participant Code across both the 2016 and 2017 periods when the Outage Reason is set to Forced, and determining the percentage of energy lost due to forced outages for each Facility Code.

In [179]:
%%sql
WITH Forced_Energy_Loss AS 
(
SELECT
    ROUND(AVG(Energy_Lost_MW),2) AS Avg_Energy_Lost,
    ROUND(SUM(Energy_Lost_MW),2) AS Total_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved' 
AND Outage_Reason = 'Forced'
GROUP BY
    Participant_Code,
    Facility_Code,
    Year 
ORDER BY Total_Energy_Lost DESC
)

SELECT 
    Avg_Energy_Lost,
    Total_Energy_Lost,
    ROUND((Total_Energy_Lost)/(SELECT SUM(Total_Energy_Lost) FROM Forced_Energy_Loss)*100,2) AS Pct_Energy_Loss,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM Forced_Energy_Loss 
GROUP BY
    Participant_Code,
    Facility_Code,
    Year 
ORDER BY Total_Energy_Lost DESC
LIMIT 5;
    

 * sqlite:///temp_db_file.db
Done.


Avg_Energy_Lost,Total_Energy_Lost,Pct_Energy_Loss,Outage_Reason,Participant_Code,Facility_Code,Year
44.16,21639.55,14.21,Forced,AURICON,AURICON_PNJ_U1,2017
85.14,19326.56,12.69,Forced,GW,BW1_GREENWATERS_G2,2017
49.69,15751.38,10.34,Forced,GW,BW1_GREENWATERS_G2,2016
87.71,13771.07,9.04,Forced,MELK,MELK_G7,2016
51.42,10696.28,7.02,Forced,AURICON,AURICON_PNJ_U1,2016


AURICON, MELK and GW participants have contributed the most to the Energy Lost due to Forced Outages

<b> Identifying top 3 participants by Total Energy Loss, and identifying the reason associated with the Total Energy Lost.

In [199]:
%%sql
SELECT * FROM
(SELECT 
    Participant_Code,
    Facility_Code,
    Description_Of_Outage,
    ROUND(SUM(Energy_Lost_MW),2) AS Total_Energy_Lost,
    ROUND(SUM(Energy_Lost_MW)*1.0/(SELECT SUM(Energy_Lost_MW) FROM AEMR_Outage_Table WHERE Status = 'Approved' AND Outage_Reason = 'Forced')*100,2) AS Pct_Energy_Loss,    
    RANK() OVER (PARTITION BY Participant_Code, Facility_Code ORDER BY Sum(Energy_Lost_MW) DESC) AS Ranking
FROM AEMR_Outage_Table
WHERE Status = 'Approved' AND Participant_Code IN ('AURICON','GW','MELK') AND Outage_Reason = 'Forced'
GROUP BY
    Participant_Code,
    Facility_Code,
    Description_Of_Outage
ORDER BY Participant_Code, Total_Energy_Lost DESC)
Where Ranking = 1;

 * sqlite:///temp_db_file.db
Done.


Participant_Code,Facility_Code,Description_Of_Outage,Total_Energy_Lost,Pct_Energy_Loss,Ranking
AURICON,AURICON_PNJ_U1,Full unit trip,6033.87,3.96,1
GW,BW1_GREENWATERS_G2,Operational Issues caused real time forced outage.,28687.54,18.84,1
MELK,MELK_G7,Safety Issues,1100.0,0.72,1
